# Week 2 - Challenge
## LangChain Practice

### Advanced Customer Service Agent with LangChain

> Deliverables

>> 1. Implementation using LangChain Expression Language (LCEL) to build the agent.
>> 2. Pydantic models for QueryAnalysis and ConversationSummary fully integrated into the chain.
>> 3. Usage examples for each suggested test query, demonstrating the end-to-end functionality.
>> 4. Results analysis. Include a screenshot or public link to a LangSmith trace for one of the complex queries (e.g.: "Urgent-Negative") to demonstrate successful tracing.

#### Setup & API Configuration

In [ ]:
# Install necessary packages

!pip install openai
!pip install python-dotenv
!pip install -qU \
  langchain-core \
  langchain-openai \
  langchain-community

In [12]:
# Import necessary libraries

import os
import openai
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from pydantic import BaseModel, Field

# Load environment variables from .env file

load_dotenv()
openai.api_key  = os.getenv('OPENAI_API_KEY')
client = openai.OpenAI()

In [13]:
openai_model = "gpt-4o-mini"

#### 1° Component - Query Analysis & Classification

Analyze the initial user query and extract key information into a structured format.

In [ ]:
from pydantic import BaseModel, Field
from typing import Literal, Optional

class ExtractedEntities (BaseModel):
    product_name: Optional[str] = Field(None, description="The specific product mentioned by the user")
    order_number: Optional[str] = Field(None, description="The order number mentioned by the user")

class QueryAnalysis (BaseModel):
    """Analyzes and classifies a customer query."""
    query_category: Literal["technical_support","billing","returns","product_inquiry","general_information"]
    urgency_level: Literal["low","medium","high"]
    customer_sentiment: Literal["positive","neutral","negative"]
    entities: ExtractedEntities    

#### 2° Component - Dynamic Response Generation

Generate a context-aware, personalized response.

#### 3° Component - Conversation Summarization & Persistance

Generate a structured summary of the entire interaction, ready for logging.

In [ ]:
from pydantic import BaseModel
from typing import List

class ConversationSummary(BaseModel):
    """A structured summary of the customer service interaction."""
    timestamp: str
    customer_id: str = "auto_generated"
    conversation_summary: str = Field(description="A concise, one-sentence summary of the interaction.")
    query_category: str
    customer_sentiment: str
    urgency_level: str
    mentioned_products: List[str]
    extracted_information: dict
    resolution_status: Literal["resolved", "pending", "escalated"]
    actions_taken: List[str] = Field(description="A list of actions the agent took or suggested.")
    follow_up_required: bool

#### Test Queries

1. Neutral-Informative: 

    "Hello, I'd like to know if you have the new iPhone 15 in stock and how much shipping costs to Chicago" 

2. Urgent-Negative: 

    "This is an emergency! My order #TEC-2024-001 never arrived and I need that laptop for work tomorrow!"

3. Satisfied-Positive: 

    "Thank you so much for the excellent service with my previous purchase. I want to buy gaming headphones"

4. Frustrated-Technical: 

    "I can't configure the router I bought last week, I've tried everything and it doesn't work"

5. Formal-Billing: 

    "Good morning, I need the receipt for my purchase from December 15th, order #TEC-2023-089"

6. Warranty-Query: 

    "I bought a tablet 8 months ago and now it won't turn on, how do I use the warranty?"